In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
import re
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed
from IPython.display import display, IFrame
from bs4 import BeautifulSoup
from pathlib import Path
from io import StringIO
from warnings import simplefilter
from math import ceil, floor

In [2]:
from sysidentpy.model_structure_selection import FROLS
from sysidentpy.model_structure_selection import AOLS
from sysidentpy.model_structure_selection import MetaMSS
from sysidentpy.basis_function import Polynomial
from sysidentpy.utils.plotting import plot_results
from sysidentpy.neural_network import NARXNN
from sysidentpy.metrics import mean_squared_error

from sktime.datasets import load_airline
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed

In [3]:
simplefilter("ignore", FutureWarning)
simplefilter("ignore", UserWarning)
simplefilter("ignore", pd.errors.PerformanceWarning)
simplefilter("ignore", RuntimeWarning)

In [4]:
# Show many outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
def get_data_list(data_folder, extension=False, pattern=["\.csv$", "\.xlsx$"]):
    github_url = f"https://github.com/munizrodrigo/datathons-time6-dados/tree/main/{data_folder}"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)

    soup = BeautifulSoup(result.text, "html.parser")
    files = []
    for ftype in pattern:
        files.extend(soup.find_all(title=re.compile(ftype)))

    filename = []
    for file in files:
        name = file.extract().get_text()
        if not extension:
            name = Path(name).resolve().stem
        filename.append(name)
    
    return filename

In [6]:
def get_temperature(city, show=False):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/temperatura_sp/{city}.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    temperature_df = pd.read_csv(csv_string, delimiter=";")
    temperature_df["din_medicao"] = pd.to_datetime(temperature_df["din_medicao"])
    temperature_df = temperature_df[temperature_df["id_varmeteo"] == "TEM_MAX"]
    temperature_df = temperature_df[["din_medicao", "val_medicao"]]
    temperature_df = temperature_df[temperature_df["val_medicao"].notnull()]
    temperature_df = temperature_df.sort_values(by="din_medicao")
    temperature_fig = px.line(temperature_df, x='din_medicao', y="val_medicao")
    if show:
        display(temperature_df)
        display(temperature_fig)
    return temperature_df

In [7]:
city_temperature = interactive(get_temperature, city=get_data_list("temperatura_sp"), show=fixed(True))

display(city_temperature)

interactive(children=(Dropdown(description='city', options=('Ariranha', 'Avare', 'Barra Bonita', 'Barra do Tur…

In [8]:
def get_meteorology(city, variable, show=False):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/meteorologia_sp/{city}.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    meteorology_df = pd.read_csv(csv_string, delimiter=";")
    meteorology_df["Hora Medicao"] = meteorology_df["Hora Medicao"] / 100
    meteorology_df["Hora Medicao"] = meteorology_df["Data Medicao"].astype(str) + " " + meteorology_df["Hora Medicao"].astype(int).astype(str) + ":00:00"
    meteorology_df["Hora Medicao"] = pd.to_datetime(meteorology_df["Hora Medicao"])
    meteorology_df = meteorology_df[["Hora Medicao", "PRECIPITACAO TOTAL, HORARIO(mm)", "UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)", "VENTO, VELOCIDADE HORARIA(m/s)"]]
    meteorology_df = meteorology_df[meteorology_df["PRECIPITACAO TOTAL, HORARIO(mm)"].notnull()]
    meteorology_df = meteorology_df[meteorology_df["UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)"].notnull()]
    meteorology_df = meteorology_df[meteorology_df["VENTO, VELOCIDADE HORARIA(m/s)"].notnull()]
    meteorology_df = meteorology_df.sort_values(by="Hora Medicao")
    variable_map = {
        "Precipitacao Total": "PRECIPITACAO TOTAL, HORARIO(mm)",
        "Umidade Relativa": "UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)",
        "Velocidade do Vento": "VENTO, VELOCIDADE HORARIA(m/s)"
    }
    variable_name = variable_map[variable]
    meteorology_df = meteorology_df[["Hora Medicao", variable_name]]
    meteorology_fig = px.line(meteorology_df, x="Hora Medicao", y=variable_name)
    if show:
        display(meteorology_df)
        display(meteorology_fig)
    return meteorology_df

In [9]:
city_meteorology = interactive(get_meteorology, city=get_data_list("meteorologia_sp"), variable=["Precipitacao Total", "Umidade Relativa", "Velocidade do Vento"], show=fixed(True))

display(city_meteorology)

interactive(children=(Dropdown(description='city', options=('Ariranha', 'Avare', 'Barra Bonita', 'Barra do Tur…

In [10]:
def get_load(utility, show=False):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/carga_sp/{utility}.xlsx"
    github_url = github_url.replace(" ", "%20")
    load_df = pd.read_excel(github_url)
    load_df["din_ocorrencia"] = pd.to_datetime(load_df["din_ocorrencia"])
    load_df = load_df[["din_ocorrencia", "val_itemserieoriginal"]]
    load_df = load_df[load_df["val_itemserieoriginal"].notnull()]
    load_df = load_df.sort_values(by="din_ocorrencia")
    load_fig = px.line(load_df, x="din_ocorrencia", y="val_itemserieoriginal")
    if show:
        display(load_df)
        display(load_fig)
    return load_df

In [11]:
utility_load = interactive(get_load, utility=get_data_list("carga_sp"), show=fixed(True))

display(utility_load)

interactive(children=(Dropdown(description='utility', options=('cpfl-paulista', 'cpfl-piratininga', 'cpfl-stac…

In [12]:
map_url = "https://www.google.com/maps/d/embed?mid=1kPQFq3OwYmW0bN-WeRcOi97vUW0ARPI&ehbc=2E312F"
IFrame(map_url, width=900, height=650)

In [13]:
def get_all_data_utility(utility, normalize=False, show=False):
    cities = get_data_list(f"distribuidoras_municipios_sp/{utility}")
    
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/distribuidoras_municipios_sp/{utility}/data_temperatura.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    
    utility_df = pd.read_csv(csv_string, delimiter=";")
    
    utility_df["data"] = pd.to_datetime(utility_df["data"])
    
    num_nan = utility_df.isna().sum() / len(utility_df.index)
    
    for column in utility_df: # Limitar NaN em 15%
        if column != "data" and column != "Carga":
            if num_nan[column] > 0.15:
                del utility_df[column]
    
    utility_df = utility_df.dropna()
    
    utility_df = utility_df.sort_values(by="data")
    
    if normalize:
        utility_df[utility_df.columns.difference(["data"])] = (utility_df[utility_df.columns.difference(["data"])]-utility_df[utility_df.columns.difference(["data"])].mean())/utility_df[utility_df.columns.difference(["data"])].std()
    
    headers = list(utility_df.columns.values)
    headers.remove("data")
    
    utility_fig = px.line(utility_df, x="data", y=headers)
    
    if show:
        display(utility_df)
        display(utility_fig)
    
    return utility_df

In [14]:
utility_df = interactive(get_all_data_utility, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=True, show=fixed(True))
display(utility_df)

interactive(children=(Dropdown(description='utility', options=('CPFL Paulista', 'CPFL Piratininga', 'CPFL Sant…

In [15]:
loss = mean_squared_error

In [16]:
def predict_frols(utility):
    df = get_all_data_utility(utility)
    df = df[["data", "Carga"]]
    df.rename(columns = {"data": "ds", "Carga": "y"}, inplace = True)
    display(df)

    train_percent = 0.8
    test_percent = 0.2

    train_count = floor(train_percent * len(df.index))
    test_count = ceil(test_percent * len(df.index))

    df_train, df_val = df.iloc[:train_count, :], df.iloc[train_count:, :]

    x_train = df_train["ds"].dt.hour.values.reshape(-1, 1)
    x_test = df_val["ds"].dt.hour.values.reshape(-1, 1)

    y = df["y"].values.reshape(-1, 1)
    y_train = df_train["y"].values.reshape(-1, 1)
    y_test = df_val["y"].values.reshape(-1, 1)

    basis_function = Polynomial(degree=1)

    sysidentpy_frols = FROLS(
        order_selection=True,
        info_criteria="bic",
        estimator="least_squares",
        basis_function=basis_function
    )

    sysidentpy_frols.fit(X=x_train, y=y_train)

    x_test = np.concatenate([x_train[-sysidentpy_frols.max_lag:], x_test])
    y_test = np.concatenate([y_train[-sysidentpy_frols.max_lag:], y_test])

    yhat = sysidentpy_frols.predict(X=x_test, y=y_test, steps_ahead=1)
    sysidentpy_loss = loss(pd.Series(y_test.flatten()[sysidentpy_frols.max_lag:]), pd.Series(yhat.flatten()[sysidentpy_frols.max_lag:]))

    n_points = 500
    plot_results(y=y_test[-n_points:], yhat=yhat[-n_points:], n=n_points, figsize=(18, 8))

In [17]:
df = interactive(predict_frols, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=fixed(False), show=fixed(False))
display(df)

interactive(children=(Dropdown(description='utility', options=('CPFL Paulista', 'CPFL Piratininga', 'CPFL Sant…

In [18]:
def predict_metamss(utility):
    df = get_all_data_utility(utility)
    df = df[["data", "Carga"]]
    df.rename(columns = {"data": "ds", "Carga": "y"}, inplace = True)
    display(df)

    train_percent = 0.8
    test_percent = 0.2

    train_count = floor(train_percent * len(df.index))
    test_count = ceil(test_percent * len(df.index))

    df_train, df_val = df.iloc[:train_count, :], df.iloc[train_count:, :]

    x_train = df_train["ds"].dt.hour.values.reshape(-1, 1)
    x_test = df_val["ds"].dt.hour.values.reshape(-1, 1)

    y = df["y"].values.reshape(-1, 1)
    y_train = df_train["y"].values.reshape(-1, 1)
    y_test = df_val["y"].values.reshape(-1, 1)

    basis_function = Polynomial(degree=1)

    sysidentpy_metamss = MetaMSS(
        basis_function=basis_function,
        estimator="least_squares",
        steps_ahead=1,
        n_agents=15,
        random_state=42
    )

    sysidentpy_metamss.fit(X_train=x_train, X_test=x_test, y_train=y_train, y_test=y_test)

    x_test = np.concatenate([x_train[-sysidentpy_metamss.max_lag:], x_test])
    y_test = np.concatenate([y_train[-sysidentpy_metamss.max_lag:], y_test])

    yhat = sysidentpy_metamss.predict(X_test=x_test, y_test=y_test, steps_ahead=1)

    metamss_loss = loss(
        pd.Series(y_test.flatten()[sysidentpy_metamss.max_lag:]),
        pd.Series(yhat.flatten()[sysidentpy_metamss.max_lag:])
    )
    
    n_points = 500
    plot_results(y=y_test[:n_points], yhat=yhat[:n_points], n=n_points, figsize=(18, 8))

In [19]:
df = interactive(predict_metamss, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=fixed(False), show=fixed(False))
display(df)

interactive(children=(Dropdown(description='utility', options=('CPFL Paulista', 'CPFL Piratininga', 'CPFL Sant…

In [20]:
def predict_aols(utility):
    df = get_all_data_utility(utility)
    df = df[["data", "Carga"]]
    df.rename(columns = {"data": "ds", "Carga": "y"}, inplace = True)
    display(df)

    train_percent = 0.8
    test_percent = 0.2

    train_count = floor(train_percent * len(df.index))
    test_count = ceil(test_percent * len(df.index))

    df_train, df_val = df.iloc[:train_count, :], df.iloc[train_count:, :]

    x_train = df_train["ds"].dt.hour.values.reshape(-1, 1)
    x_test = df_val["ds"].dt.hour.values.reshape(-1, 1)

    y = df["y"].values.reshape(-1, 1)
    y_train = df_train["y"].values.reshape(-1, 1)
    y_test = df_val["y"].values.reshape(-1, 1)

    basis_function = Polynomial(degree=1)

    sysidentpy_AOLS = AOLS(
        basis_function=basis_function
    )
    sysidentpy_AOLS.fit(X=x_train, y=y_train)

    x_test = np.concatenate([x_train[-sysidentpy_AOLS.max_lag:], x_test])
    y_test = np.concatenate([y_train[-sysidentpy_AOLS.max_lag:], y_test])

    yhat = sysidentpy_AOLS.predict(X=x_test, y=y_test, steps_ahead=1)
    aols_loss = loss(pd.Series(y_test.flatten()[sysidentpy_AOLS.max_lag:]), pd.Series(yhat.flatten()[sysidentpy_AOLS.max_lag:]))

    n_points = 500
    plot_results(y=y_test[-n_points:], yhat=yhat[-n_points:], n=n_points, figsize=(18, 8))

In [21]:
df = interactive(predict_aols, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=fixed(False), show=fixed(False))
display(df)

interactive(children=(Dropdown(description='utility', options=('CPFL Paulista', 'CPFL Piratininga', 'CPFL Sant…

In [24]:
def predict_neuralprophet(utility):
    set_random_seed(1)

    df = get_all_data_utility(utility)
    df = df[["data", "Carga"]]
    df.rename(columns = {"data": "ds", "Carga": "y"}, inplace = True)
    display(df)

    m = NeuralProphet(
        n_lags=10,
        n_forecasts=1,
        num_hidden_layers=2,
        d_hidden=30,
        learning_rate=0.004
    )

    metrics = m.fit(df, freq="H")

    df_train, df_val = m.split_df(df, valid_p=0.15)

    m.test(df_val)

    future = m.make_future_dataframe(df_val, n_historic_predictions=True)
    forecast = m.predict(future)

    plt.figure(figsize=(18, 8))
    plt.plot(forecast["y"][-1000:], "ro-")
    plt.plot(forecast["yhat1"][-1000:], "k*-")

In [25]:
df = interactive(predict_neuralprophet, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=fixed(False), show=fixed(False))
display(df)

interactive(children=(Dropdown(description='utility', options=('CPFL Paulista', 'CPFL Piratininga', 'CPFL Sant…

In [ ]:
def predict_multiprophet(utility):
    set_random_seed(1)

    df = get_all_data_utility(utility)
    df = df[["data", "Carga"]]
    df.rename(columns = {"data": "ds", "Carga": "y"}, inplace = True)
    display(df)

    m = NeuralProphet(
        n_lags=10,
        n_forecasts=1,
        num_hidden_layers=2,
        d_hidden=30,
        learning_rate=0.004
    )

    metrics = m.fit(df, freq="H")

    df_train, df_val = m.split_df(df, valid_p=0.15)

    m.test(df_val)

    future = m.make_future_dataframe(df_val, n_historic_predictions=True)
    forecast = m.predict(future)

    plt.figure(figsize=(18, 8))
    plt.plot(forecast["y"][-1000:], "ro-")
    plt.plot(forecast["yhat1"][-1000:], "k*-")

In [ ]:
df = interactive(predict_multiprophet, utility=get_data_list("distribuidoras_municipios_sp", pattern=["CPFL", "EDP", "Elektro", "Eletropaulo", "Energisa"]), normalize=fixed(False), show=fixed(False))
display(df)